# Diabetes Classification using Dense Layers

In [45]:
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score


In [46]:
df = pd.read_csv('data/diabetes.csv')

X = df.drop('Outcome', axis=1)
y = df['Outcome']

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [48]:
X_train_tensor = torch.FloatTensor(X_train.values)
y_train_tensor = torch.FloatTensor(y_train.values)
X_test_tensor = torch.FloatTensor(X_test.values)
y_test_tensor = torch.FloatTensor(y_test.values)

In [49]:
class SimpleDenseNet(nn.Module):
    def __init__(self, input_dim):
        super(SimpleDenseNet, self).__init__()
        self.linear = nn.Linear(input_dim, 1)
    
    def forward(self, x):
        return torch.sigmoid(self.linear(x))

In [50]:
net = SimpleDenseNet(X_train.shape[1])
print(net)

SimpleDenseNet(
  (linear): Linear(in_features=8, out_features=1, bias=True)
)


In [51]:
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001)

In [53]:
num_epochs = 1000
batch_size = 32

for epoch in range(num_epochs):
    for i in range(0, len(X_train_tensor), batch_size):
        batch_X = X_train_tensor[i:i+batch_size]
        batch_y = y_train_tensor[i:i+batch_size]
        
        outputs = net(batch_X)
        loss = criterion(outputs.squeeze(), batch_y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [100/1000], Loss: 1.5320
Epoch [200/1000], Loss: 1.5061
Epoch [300/1000], Loss: 1.6007
Epoch [400/1000], Loss: 1.2742
Epoch [500/1000], Loss: 1.4674
Epoch [600/1000], Loss: 1.0060
Epoch [700/1000], Loss: 1.0148
Epoch [800/1000], Loss: 0.9939
Epoch [900/1000], Loss: 0.9836
Epoch [1000/1000], Loss: 0.9763


In [54]:
net.eval()
with torch.no_grad():
    y_pred = net(X_test_tensor)
    y_pred_class = (y_pred.squeeze() > 0.5).float()
    accuracy = accuracy_score(y_test, y_pred_class)
    print(f'Accuracy: {accuracy:.4f}')
    print('Confusion Matrix:')
    print(confusion_matrix(y_test, y_pred_class))
    

Accuracy: 0.6818
Confusion Matrix:
[[92  7]
 [42 13]]
